In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('../RotationalEnsemble/spx_test.csv')

steps of a genetic algorithm:

initialization
evaluation
elitism
parent selection
reproduction
mutation

In [ ]:
#example for initiation and evaluation steps

import _1_initialization as _1
import _2_evaluation as _2
import _3_elitism as _3
import _4_parentselection as _4
import _5_reproduction as _5
import _6_mutation as _6
import _9_utility as util

dataset = util.load_large_csv('some file name here')
dataset, fss = util.drop_all_targets()
dataset = dataset.values

lag_allowance = 30
generations = 5
hold_for = 10


population = _1.generate_initial_population(
    sample_size=100,
    pattern_size=1,
    feat_idx_pool=fss,
    lag_allowance=lag_allowance,
    skew_lag_prob=False
)

#begin generational loop
for generation in range(generations):

	returns, kelsch_ratio = _2.fitness(
		data = dataset,
		genes= population,
		hold_for=hold_for,
		lag_allow=lag_allowance,
		specific_data='form_519'
	)

	population = _2.associate(
		genes=population,
		returns=returns,
		kelsch_ratio=kelsch_ratio
	)

	criteria = 'profit_factor'

	population = _2.sort_population(
		population=population,
		criteria=criteria
	)

	avg, top = _2.simple_generational_stat_output(population,criteria)
	print(f"Generation {generation+1} ({criteria}): AVG {avg}, BEST {top}, FROM {len(population)} GENES.")

	elites = _3.collect_elite(
		sorted_population=population,
		filter_criteria=1
	)

	parents = _4.collect_parents(
		sorted_population=population,
		criteria=criteria,
		num_parents=2
	)

	family = _5.reproduce(
		parents=parents,
		mode='exponential'
	)

	shuffled_family = _1.shuffle_population(
		population=family
	)

	mutated_family = _6.mutation_round(
		shuffled_population=shuffled_family,
		partial_mutation_prob=0.050,
		pattern_mutation_prob=0.025,
		feat_idx_pool=fss
		
	)

	population = _1.combine_populations(
		populations=[elites, mutated_family]
	)

candlestick based genalg params:
modify these based off of largely expanded featurespace

popsize - 200
num gens - 9
elitism copys - 1
mutation prob - 5%
fresh pattern prob - 2%
min number of patterns - 2.5%